# 1. 主从复制准备

一般常规主从复制在不同的机器上，并且监听的端口均为默认的3306。  
本文主从复制技术环境以单数据库多实例来讲解学习，上文已经写了怎么配置mysql多实例。

## 1.1 定义服务器角色

主库（mysql master）:ip为192.168.136.113 port为3306

从库1（mysql slave）:ip为192.168.136.113 port为3307

从库2 （mysql slave）:ip为192.168.136.113 port为3308